# Proof of concept in torch

## What we know
* Images have different size 
    -fully convolutional network or resize
    - what to do with batch size
* Extra images has no labels such as right, left and top
* Image position asinput or axuiliary output 
* Possibly use CapsNet

In [155]:
import torch 
import torch.utils.data as torch_data
import torch.nn as nn

import torchvision.io as io
import torchvision.transforms as T

import pandas as pd
import os

import numpy as np

In [36]:
# Loading data
TRAIN_CSV_PATH = "../data/train.csv"
TEST_CSV_PATH = "../data/test.csv"
EXTRA_IMAGES_PATH = "../data/extra_images.csv"
SAMPLE_SUBMISSION_PATH = "../data/sample_submission.csv"
IMAGES_PATH = "../data/images"

In [162]:
# Label map
train_csv = pd.read_csv(TRAIN_CSV_PATH)
# test_csv  = pd.read_csv(TEST_CSV_PATH)
extra_images = pd.read_csv(EXTRA_IMAGES_PATH)

all_imgs = pd.concat([train_csv, extra_images])

In [171]:
unique_ids = set(all_imgs.turtle_id)
len(unique_ids)
# filtered_set = [x for x in unique_ids if not np.isnan(x)]
all_labels = list(unique_ids)
all_labels.append("new_turtle")
len(all_labels)

2266

In [170]:
't_id_WLAIJIZ0' in unique_ids
str('nan') in unique_ids

False

In [153]:
str('nan')

'nan'

In [179]:
class TurtleDataset(torch_data.Dataset):
    
    def __init__(self, img_labels, img_dir, 
                 img_transform = None, transform_target = None):
        self.img_labels = pd.read_csv(img_labels)
        self.img_dir = img_dir
        self.img_transform = img_transform
        self.transform_target = transform_target
   
    def __len__(self):
        return len(self.img_labels)
    
    def __getitem__(self, idx):
        # Image path
        img_id = self.img_labels.iloc[idx, 0] # First col
        img_filename = img_id + ".JPG"
        img_path = os.path.join(self.img_dir, img_filename)
        
        # Reading an image
        image = io.read_image(img_path)
        image_location = self.img_labels.iloc[idx, 1]
        turtle_id = self.img_labels.iloc[idx, 2]
        
        if self.img_transform:
            image = self.img_transform(image)
        if self.transform_target:
            label = self.transform_target(label)
        
        return (image.to(torch.float) / 255).cuda(), image_location, turtle_id        

In [185]:
transform = T.Compose([
    T.Resize((500, 500))
])

turtle_dataset = TurtleDataset(
    img_labels = TRAIN_CSV_PATH,
    img_dir = IMAGES_PATH,
    img_transform = transform
)

In [186]:
img, pos, lbl = turtle_dataset[0]

In [107]:
img.dtype

torch.float32

In [187]:
train_turtle_dl = torch_data.DataLoader(turtle_dataset, batch_size=32, shuffle = True)

## Sample architecture

In [181]:
turtle_net_1 = nn.Sequential(
        nn.Conv2d(3, 10, (3, 3)),
        nn.Conv2d(10, 100, (3, 3), stride=2),
        nn.Conv2d(100, 1000, (3, 3), stride=2),
        nn.Conv2d(1000, 2266, (3, 3), stride=2)
    ).cuda()

In [189]:
sample_batch = next(iter(train_turtle_dl))

In [190]:
sample_tensor = sample_batch[0]

In [191]:
out = turtle_net_1(sample_tensor)

In [178]:
out[1].shape

torch.Size([2266, 61, 61])